In [3]:
%run 'StreamTemp-Integ.py'

optData:  OrderedDict([('name', 'hydroDL.data.camels.DataframeCamels'), ('subset', 'All'), ('varT', ['dayl(s)', 'prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)', '00060_Mean']), ('varC', ['DRAIN_SQKM', 'STREAMS_KM_SQ_KM', 'STOR_NID_2009', 'FORESTNLCD06', 'PLANTNLCD06', 'SLOPE_PCT', 'RAW_DIS_NEAREST_MAJ_DAM', 'PERDUN', 'RAW_DIS_NEAREST_DAM', 'RAW_AVG_DIS_ALL_MAJ_DAMS', 'T_MIN_BASIN', 'T_MINSTD_BASIN', 'RH_BASIN', 'RAW_AVG_DIS_ALLDAMS', 'PPTAVG_BASIN', 'HIRES_LENTIC_PCT', 'T_AVG_BASIN', 'T_MAX_BASIN', 'T_MAXSTD_BASIN', 'NDAMS_2009', 'ELEV_MEAN_M_BASIN']), ('target', ['S', 't', 'r', 'e', 'a', 'm', 'T', 'e', 'm', 'p']), ('tRange', [20101001, 20141001]), ('doNorm', [True, True]), ('rmNan', [True, False]), ('daObs', 0), ('damean', False), ('davar', 'streamflow'), ('dameanopt', 0), ('lckernel', None), ('fdcopt', False)])
TempTarget:  00010_Mean
forcing_path:  /caldera/projects/usgs/water/iidd/datasci/psu/LSTM_temp2/input/forcing/forcing_99%_days_99sites_newv1.feather
attr_path:  /

FileNotFoundError: [Errno 2] No such file or directory: '/caldera/projects/usgs/water/iidd/datasci/psu/LSTM_temp2/input/forcing/forcing_99%_days_99sites_newv1.feather'

In [ ]:
#try to recreate basinnorm.json
from hydroDL.data import camels

#forcing_data = pd.read_feather(forcing_path)
#print(forcing_data.__class__)
#print(forcing_data['site_no'].unique())
calStatAll()

In [ ]:
import torch
torch.cuda.is_available()

### Checking model outputs
Need to join on site_no column from attributes file for each set of outputs
Then can compare proper 99 rows from data release

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_release_pred = np.load('../rahmani-resdag-data-release/in_data/Results/99_10/pred_2000_50_365_70_2014_2016_ens.npy')[:,:,0]
data_release_pred.shape

In [ ]:
#vector of input files
initialization_preds = ['TempDemo/FirstRun/epochs2000_batch47_rho365_hiddensize100_Tstart20101001_Tend20141001_1/All-2010-2016/pred.npy',
                       'TempDemo/FirstRun/epochs2000_batch47_rho365_hiddensize100_Tstart20101001_Tend20141001_2/All-2010-2016/pred.npy',
                       'TempDemo/FirstRun/epochs2000_batch47_rho365_hiddensize100_Tstart20101001_Tend20141001_3/All-2010-2016/pred.npy',
                       'TempDemo/FirstRun/epochs2000_batch47_rho365_hiddensize100_Tstart20101001_Tend20141001_4/All-2010-2016/pred.npy',
                       'TempDemo/FirstRun/epochs2000_batch47_rho365_hiddensize100_Tstart20101001_Tend20141001_5/All-2010-2016/pred.npy',
                       'TempDemo/FirstRun/epochs2000_batch47_rho365_hiddensize100_Tstart20101001_Tend20141001_6/All-2010-2016/pred.npy']

#use length of vector to assign each to an array index
def load_initializations(files):
    init_array = []
    for i in range(len(files)):
        print(files[i])
        init_array.append(np.load(files[i])[:,:,0])
    return(init_array)
        
init_array = load_initializations(initialization_preds)
len(init_array)

In [ ]:
init_means = np.mean(init_array, axis = 0)
init_means.shape

In [ ]:
this_run_pred = np.load('TempDemo/FirstRun/epochs2000_batch47_rho365_hiddensize100_Tstart20101001_Tend20141001_1/All-2010-2016/pred.npy')[:,:,0]
this_run_pred2 = np.load('TempDemo/FirstRun/epochs2000_batch47_rho365_hiddensize100_Tstart20101001_Tend20141001_2/All-2010-2016/pred.npy')[:,:,0]
this_run_pred_mean = np.mean([this_run_pred, this_run_pred2], axis = 0)
this_run_pred_mean.shape

In [ ]:
attr = pd.read_feather('scratch/SNTemp/Forcing/attr_new/attr_temp99%_days_99sites_newv1.feather')
attr_data_release = pd.read_feather('../rahmani-resdag-data-release/in_data/Forcings_attr/attr_temp10__days_415sites.feather')
data_release_rows = attr_data_release['site_no'].isin(attr['site_no'])

In [ ]:
data_release_pred_filtered = data_release_pred[data_release_rows, :]
plt.matshow(data_release_pred_filtered)
plt.colorbar()
plt.show()

In [ ]:
#compare predictions
this_run_pred_mean.shape
plt.matshow(this_run_pred_mean)
plt.colorbar()
plt.show()

In [ ]:
diff = this_run_pred_mean - data_release_pred_filtered
diff.shape
plt.matshow(diff)
plt.colorbar()
plt.show()

In [ ]:
np.sqrt(np.mean(diff ** 2))

In [ ]:
#what is difference between random initializations?
diff_inits = init_array[5] - init_array[1]
diff.shape
plt.matshow(diff_inits)
plt.colorbar()
plt.show()
np.sqrt(np.mean(diff_inits ** 2))

In [ ]:
#get range of diffs across initializations
for init in init_array:
    diff = init - init_array[5] #or swap in data_release_pred_filtered to compare to data release
    print(np.sqrt(np.mean(diff ** 2)))

In [ ]:
#Is there any bias relative to data release?
for init in init_array:
    diff = init - data_release_pred_filtered #or swap in data_release_pred_filtered to compare to data release
    print(np.mean(diff))

In [ ]:
#what are RMSE compared to observations?
obs = np.load('TempDemo/FirstRun/epochs2000_batch47_rho365_hiddensize100_Tstart20101001_Tend20141001_1/All-2010-2016/obs.npy')[:,:,0]
obs.shape
for init in init_array:
    diff = init - obs #or swap in data_release_pred_filtered to compare to data release
    print(np.sqrt(np.nanmean(diff ** 2)))

print("RMSE of mean of 6:")
diff_obs_mean = np.mean(init_array, axis = 0) - obs
print(np.sqrt(np.nanmean(diff_obs_mean ** 2)))
print("RMSE of data release equivalent")
diff_data_release = data_release_pred_filtered - obs
print(np.sqrt(np.nanmean(diff_data_release ** 2)))

In [ ]:
#investigate Statistics_basinnorm.json and how it is created
attr = pd.read_feather('scratch/SNTemp/Forcing/attr_new/attr_temp99%_days_99sites_newv1.feather')


In [ ]:
attr

In [ ]:
import os
os.environ['PROJ_LIB'] = r'/root/miniconda3/share/proj/'
from mpl_toolkits import basemap